In [1]:
!pip install --upgrade git+https://github.com/tusharsarkar3/XBNet.git

  Cloning https://github.com/tusharsarkar3/XBNet.git to c:\users\leepi\appdata\local\temp\pip-req-build-ea6ttoga
  Resolved https://github.com/tusharsarkar3/XBNet.git to commit 2b771d6beb9f15ae48e2a365f46d6f996811086f
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached matplotlib-3.4.3-cp39-cp39-win_amd64.whl (7.1 MB)
  Created wheel for XBNet: filename=XBNet-1.4.6-py3-none-any.whl size=13303 sha256=c091f924bec8c1c1e32f305c040fc8b32ba1535e748ce626b9ae4d2963a92537
  Stored in directory: C:\Users\leepi\AppData\Local\Temp\pip-ephem-wheel-cache-opahotkh\wheels\e1\fc\fd\454f766aabc107e8f137228c629835b432808c0ce974eff641
Successfully built XBNet
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1
  Attempting uninstall: sklearn
    Found existing installation: sklear

  Running command git clone --filter=blob:none --quiet https://github.com/tusharsarkar3/XBNet.git 'C:\Users\leepi\AppData\Local\Temp\pip-req-build-ea6ttoga'


In [1]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from XBNet.training_utils import training,predict
from XBNet.models import XBNETClassifier
from XBNet.run import run_XBNET

import glob
from pandas.core.common import flatten
import random

In [2]:
# set path
split = 'split1'
path = "../DATA/"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/{}/test/test_{}.csv".format(path,split,split)
valid_path = "{}/{}/val/val_{}.csv".format(path,split,split)

In [3]:
print(train_path)
print(test_path)
print(valid_path)

../DATA//split1/train/train_split1.csv
../DATA//split1/test/test_split1.csv
../DATA//split1/val/val_split1.csv


In [4]:
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)

(292, 32)
(292, 30)
['B' 'M']


In [5]:
# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[2:]]
print(x_test.shape)
y_test = test_data[test_data.columns[1]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)

(91, 32)
(91, 30)
['B' 'M']


In [6]:
# Load the validation set
val_data = pd.read_csv(valid_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(72, 32)
(72, 30)
['B' 'M']


In [7]:
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()

In [10]:
from sklearn.model_selection import train_test_split

print("Shape of training set:", x_train.shape)
print("Shape of test set:", x_test.shape)

Shape of training set: (292, 30)
Shape of test set: (91, 30)


In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

In [13]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

In [15]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

parameters = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : range(2, 32, 1),
    'min_samples_leaf' : range(1, 10, 1),
    'min_samples_split' : range(2, 10, 1),
    'splitter' : ['best', 'random']
}

grid_search_dt = GridSearchCV(dtc, parameters, cv = 5, n_jobs = -1, verbose = 1)
grid_search_dt.fit(x_train, y_train)

Fitting 5 folds for each of 8640 candidates, totalling 43200 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 32),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'splitter': ['best', 'random']},
             verbose=1)

In [16]:
# best parameters

grid_search_dt.best_params_

{'criterion': 'gini',
 'max_depth': 31,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'splitter': 'random'}

In [17]:
grid_search_dt.best_score_

0.986323787258913

In [19]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 28, min_samples_leaf = 1, min_samples_split = 8, splitter = 'random')
dtc.fit(x_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=28, min_samples_split=8,
                       splitter='random')

In [20]:
y_pred = dtc.predict(x_test)

In [22]:
# accuracy score

print(accuracy_score(y_train, dtc.predict(x_train)))

dtc_acc = accuracy_score(y_test, dtc.predict(x_test))
print(dtc_acc)

1.0
0.9340659340659341


In [23]:
# confusion matrix

print(confusion_matrix(y_test, y_pred))

[[50  2]
 [ 4 35]]


In [24]:
# classification report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94        52
           1       0.95      0.90      0.92        39

    accuracy                           0.93        91
   macro avg       0.94      0.93      0.93        91
weighted avg       0.93      0.93      0.93        91

